# Baseline CAD1 Task1

This tutorial walks through the process of running the CAD1 Task1 baseline using the shell Interface


## Setting the Location of the Project

For convenience, we are setting an environment variable with the location of the root working directory of the project. This variable will be used in various places throughout the tutorial. Please change this value to reflect where you have installed this notebook on your system.

In [1]:
import os
os.environ["NBOOKROOT"] = os.getcwd()
os.getenv("NBOOKROOT")

'/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines'

## Cloning the Clarity Repository
We first need to install the Clarity package.

In [2]:
print("Cloning git repo...")
!git clone --quiet https://github.com/claritychallenge/clarity.git
%cd clarity
%pip install -e .
import os
import sys

sys.path.append(f'{os.getenv("NBOOKROOT")}/clarity')
from IPython.display import clear_output

clear_output()
print("Repository installed")

Repository installed


## Get the demo data

We will be using music audio and listener metadata.

In [6]:
!gdown 10SfuZR7yVlVO6RwNUc3kPeJHGiwpN3VS
!mv cadenza_data_demo.tar.xz recipes/cad1/task1/baseline
%cd recipes/cad1/task1/baseline
!tar -xvf cadenza_data_demo.tar.xz

clear_output()
print("Data installed")

Data installed


## Changing working Directory

Next, we change working directory to the location of the shell scripts we wish to run.

In [5]:
%cd {os.environ['NBOOKROOT']}/clarity/recipes/cad1/task1/baseline

/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad1/task1/baseline


## Inspecting Existing Configuration

Let's inspect the `config.yml` file.

```yaml
path:
  root: ../../cadenza_data_demo/cad1/task1
  metadata_dir: ${path.root}/metadata
  music_dir: ${path.root}/audio/musdb18hq
  music_file: ${path.metadata_dir}/musdb18.valid.json
  listeners_file: ${path.metadata_dir}/listeners.valid.json
  music_segments_test_file: ${path.metadata_dir}/musdb18.segments.test.json
  exp_folder: ./exp_${separator.model} # folder to store enhanced signals and final results

team_id: T001

sample_rate: 44100       # sample rate of the input mixture
stem_sample_rate: 24000  # sample rate output stems
remix_sample_rate: 32000 # sample rate for output remixed signal

nalr:
  nfir: 220
  sample_rate: ${sample_rate}

apply_compressor: False
compressor:
  threshold: 0.35
  attenuation: 0.1
  attack: 50
  release: 1000
  rms_buffer_size: 0.064

soft_clip: True

separator:
  model: demucs   # demucs or openunmix
  device: ~

evaluate:
  set_random_seed: True
  small_test: False
  batch_size: 1  # Number of batches
  batch: 0       # Batch number to evaluate

# hydra config
hydra:
  run:
    dir: ${path.exp_folder}
```

## Running Enhancement 

We are now ready to run the `enhance.py` python script. However, the standard configuration is designed to work with the full dataset. We can redirect the script to the correct folders by overriding the appropriate configuration parameters.

In [13]:
!python enhance.py path.root={os.environ['NBOOKROOT']}/clarity/recipes/cad1/task1/baseline/cadenza_data_demo/cad1/task1

/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad1/task1/baseline/evaluate.py:155: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")
/home/gerardoroadabike/Extended/Projects/cadenza_tutorials/baselines/clarity/recipes/cad1/task1/baseline/enhance.py:350: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")
/home/gerardoroadabike/anaconda3/envs/clarity39/lib/python3.10/site-packages/hydra/_internal/hydra.py:119: UserWarning: Future Hydra versions will no longer change working directory at job runtime by default.
See https://hydra.cc/docs/1.2/upgrades/1.1_to_1.2/changes_to_job_working_dir/ for more information.
  ret = run_job(
[2024-08-21 10:05:38,435]

Now we have the enhanced output. Below, we can load and play the audios to listen the results.

In [ ]:
from pathlib import Path
from scipy.io import wavfile

import IPython.display as ipd

audio_path = Path("exp_demucs/enhanced_signals")
audio_files = [f for f in audio_path.glob('*/*/*') if f.suffix == '.wav']

for file_to_play in audio_files:
  sample_rate, signal = wavfile.read(file_to_play)
  # Take only 30 seconds
  if signal.shape[-1] == 1:
    signal = signal[30*sample_rate: 60*sample_rate, :]
  else:
    signal = signal[30*sample_rate: 60*sample_rate]
  print(file_to_play.name)
  ipd.display(ipd.Audio(signal.T, rate=sample_rate))

Now that we have enhanced audios we can use the evaluate the outputs by using the HAAQI metric. The evaluation is run in the same manner as the enhancement script.

In [ ]:
!python evaluate.py path.root={os.environ['NBOOKROOT']}/clarity/recipes/cad1/task1/baseline/cadenza_data_demo/cad1/task1